In [19]:
# Further Cleaning and Processing data.csv
'''
import pandas as pd
import numpy as np
df = pd.read_csv("cleanedData.csv")
df_lectures = df[df['CourseTitle'].notna()]
locations_to_drop = ['T B A', '', 'ON   ASYNC', 'ON   LINE']
df_lectures = df_lectures[~df_lectures.isin(locations_to_drop).any(axis=1)]
room_df = pd.read_csv("roomToBuilding.csv")
coordinate_df = pd.read_csv("coordinates.csv")
data = pd.merge(df_lectures, room_df, on='Location')
data = pd.merge(data, coordinate_df, on='Location')
data.rename(columns={'Department': 'DepartmentCode'}, inplace=True)
depNames = pd.read_csv("depNames.csv")
data = pd.merge(data, depNames, on='DepartmentCode', how='left')
data = data.iloc[:, [0, 1, -1, 2, 3, 4, 5, 6, 7]]
data['Longitude'] = ""
data['Latitude'] = ""
for r in range(len(data)):
    coord = data.loc[r, "coordinates"].split(',')
    data.at[r, 'Latitude'] = float(coord[0])
    data.at[r, 'Longitude'] = float(coord[1])
'''
data = pd.read_csv("cleanedData.csv")

In [20]:
# Creating and Processing Map Data
mapData = pd.DataFrame(columns=['Department','building','Count','Percentage','Latitude','Longitude'])
data.drop(['Location'], axis=1)
coordinate_df['Latitude'] = ""
coordinate_df['Longitude'] = ""
for r in range(len(coordinate_df)):
    coord = coordinate_df.loc[r, "coordinates"].split(',')
    coordinate_df.at[r, 'Latitude'] = float(coord[0])
    coordinate_df.at[r, 'Longitude'] = float(coord[1])

for d in depNames['DepartmentName']:
    df = data[data['DepartmentName'] == d]
    mergeDf = pd.DataFrame()
    lat = df.groupby(['Latitude']).count()
    mergeDf['Latitude'] = pd.to_numeric(lat.index, errors='coerce')
    mergeDf['Count'] = lat['CourseID'].values
    mergeDf['Department'] = np.array(d)
    mergeDf['Percentage'] = np.round(mergeDf['Count']/len(df)*100, 2)
    mergeDf = mergeDf.merge(coordinate_df.loc[:, ['Latitude', 'Longitude', 'building']].drop_duplicates(subset=['Latitude', 'building']), 
                            on="Latitude", how="left")
    mergeDf = mergeDf.iloc[:, [2, 5, 1, 3, 0, 4]]
    #mergeDf['Blurb'] = ""
    #for i in range(len(mergeDf)):
    #    mergeDf.at[i, 'Blurb'] = "Number of classes: " + str(mergeDf.at[i, 'Count']) + " (" + str(mergeDf.at[i, 'Percentage'].round())+ "2%)"
    mapData = pd.concat([mapData, mergeDf])

mapData.rename(columns={'building': 'Building'}, inplace=True)
mapData.to_csv("mapData.csv", index=False) 

mapData


KeyError: "['building'] not in index"